In [1]:
import geoplotlib
from geoplotlib.utils import read_csv, BoundingBox, DataAccessObject
import pandas as pd
import datetime
import json
from shapely.geometry import shape, Point

In [2]:
travels = pd.read_csv('travels.csv')

In [3]:
# Nome dos geojsons a serem comparados com as coordenadas
json_name = ['vancouver', 'burnaby', 'sea-island', 'new-westminster',
             'north-vancouver', 'grousewoods', 'university']
geojsons = []

In [4]:
for js in json_name:
    with open('travels_hotmap/'+js+'.json') as json_data:
        geojsons.append(json.load(json_data))

In [ ]:
# Separando os nomes dos distritos e agrupando em regioes em comum 
names = []
group = []
count = 0
for json in geojsons:
    count += 1
    for feature in json['features']:

        name = feature['properties']['name']
        
        if (name not in names):
            group.append(count)
            names.append(feature['properties']['name'])
            
names = pd.DataFrame(names, columns=['name'])
names['group'] = pd.DataFrame(group)

In [6]:
# Listando todas as possibilidades de saida e chegada

links = []

for i in range(len(names)):
    for j in range(len(names)):
        links.append([i,j])
        
links = pd.DataFrame(links, columns=['source', 'target'])
links['value'] = [0]*len(links)

In [ ]:
# Contando a quantidade de viagens para cada trajeto

for (start_lon, start_lat, end_lon, end_lat, car_id) in zip(travels['Start_lon'], travels['Start_lat'], 
                                                    travels['End_lon'], travels['End_lat'], travels['Id']):
        
    start_point = Point(start_lon, start_lat)
    end_point = Point(end_lon, end_lat)
    start = end = None
    found = False
    
    # Buscando em cada geojson se a viagem está em um certo distrito
    
    for json in geojsons:
        if (not found):
            
            for feature in json['features']:
                
                polygon = shape(feature['geometry'])
                
                if polygon.contains(start_point):
                    # Coletando o distrito de saida
                    start = names[names['name'] == feature['properties']['name']].index[0]
                    found = True
                    break
        else:
            break
                
    found = False
                
    for json in geojsons:
        if (not found):
            
            for feature in json['features']:
                
                polygon = shape(feature['geometry'])
                
                if polygon.contains(end_point):
                    # Coletando o distrito de chegada
                    end = names[names['name'] == feature['properties']['name']].index[0]
                    found = True
                    break
                    
        else:
            break
            
                    
    # Somando mais uma viagem no trajeto
    links['value'].loc[(links['source'] == start) & (links['target'] == end)] += 1

In [9]:
links.to_csv('travels_hotmap/links.csv', index=False, encoding='utf-8')

In [12]:
links = pd.read_csv('travels_hotmap/links.csv')

## Organizando os dados de viagens como um fluxo de índice da linha para o da coluna

### Fluxo de todas as interações

In [22]:
flux = pd.DataFrame()
for i in range(len(names)):
    flux[i] = [0] * len(names)

In [23]:
for i in range(len(links)):
    source = links['source'].iloc[i] 
    target = links['target'].iloc[i]
    
    flux[target].iloc[source] += links['value'].iloc[i]

In [24]:
flux.index = list(names['name'])
flux.columns = list(names['name'])

In [28]:
flux.to_csv('travels_hotmap/flux.csv', sep=' ', encoding='utf-8')

### Fluxo de viagens fora de Vancouver

In [14]:
flux = pd.DataFrame()
for i in range(names['group'].max()):
    flux[i] = [0] * names['group'].max()

In [15]:
for i in range(len(links)):
    source = names['group'].iloc[links['source'].iloc[i]] - 1
    target = names['group'].iloc[links['target'].iloc[i]] - 1
    
    flux[target].iloc[source] += links['value'].iloc[i]

In [16]:
flux.index = list(json_name)
flux.columns = list(json_name)

In [17]:
flux.to_csv('travels_hotmap/fluxOutVancouver.csv', sep=' ', encoding='utf-8')

### Fluxo de viagens dentro de Vancouver

In [18]:
districts = names[names['group'] == 1].index

In [19]:
flux = pd.DataFrame()
for i in range(len(districts)):
    flux[i] = [0] * len(districts)

In [20]:
links_districts = links[(links['source'] < len(districts)) & (links['target'] < len(districts))]
for i in range(len(links_districts)):
    source = links_districts['source'].iloc[i]
    target = links_districts['target'].iloc[i]
    
    flux[target].iloc[source] += links_districts['value'].iloc[i]

In [21]:
flux.index = list(names[names['group'] == 1]['name'])
flux.columns = list(names[names['group'] == 1]['name'])

In [22]:
flux.to_csv('travels_hotmap/fluxInVancouver.csv', sep=' ', encoding='utf-8')